# Getting Started

> An introduction to using the experimental features of Ragas

In [1]:
from ragas_experimental.utils import get_test_directory

/Users/jjmachan/workspace/eglabs/ragas/.venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
from ragas_experimental import Project

In [3]:
root_dir = get_test_directory()
root_dir

'/var/folders/2y/02fp70k56p75ldrkgtx7z10r0000gn/T/ragas_test_foqCNH4eTMFE'

In [4]:
p = Project(
    project_id="test",
    backend="local",
    root_dir=root_dir,
)

p

In [5]:
from ragas_experimental import BaseModel
import typing as t

class TestDataRow(BaseModel):
    id: t.Optional[int]
    query: str
    persona: t.List[t.Literal["opt1", "opt2", "opt3"]]

In [6]:
dataset = p.create_dataset(
    name="test_dataset",
    model=TestDataRow,
)

dataset

Dataset(name='test_dataset', model=TestDataRow, len=0)

In [7]:
for i in range(3):
    row = TestDataRow(id=i, query=f"query_{i}", persona=["opt1"])
    dataset.append(row)

dataset

Dataset(name='test_dataset', model=TestDataRow, len=3)

In [8]:
from ragas_experimental.metric import MetricResult

class ExperimentDataRow(TestDataRow):
    response: str 
    metrics: t.List[MetricResult]


@p.experiment(ExperimentDataRow, name_prefix="Workshop")
async def run_experiment(row: TestDataRow):
    response = "test"
    score1 = MetricResult(result=1, reason="score 1")
    score2 = MetricResult(result=0, reason="score 2")

    experiment_view = ExperimentDataRow(
        id=row.id,
        query=row.query,
        persona=["opt1"],
        response=response,
        metrics=[score1, score2],
    )
    print(experiment_view.model_dump())
    return experiment_view

In [9]:
await run_experiment.run_async(dataset)

Running experiment:   0%|          | 0/6 [00:00<?, ?it/s]

{'id': 1, 'query': 'query_1', 'persona': ['opt1'], 'response': 'test', 'metrics': [1, 0]}
{'id': 2, 'query': 'query_2', 'persona': ['opt1'], 'response': 'test', 'metrics': [1, 0]}
{'id': 0, 'query': 'query_0', 'persona': ['opt1'], 'response': 'test', 'metrics': [1, 0]}


ValueError: dict contains fields not in fieldnames: 'query', 'persona', 'id'

In [13]:
%debug

> /Users/jjmachan/.local/share/uv/python/cpython-3.12.8-macos-aarch64-none/lib/python3.12/csv.py(159)_dict_to_list()
    157             wrong_fields = rowdict.keys() - self.fieldnames
    158             if wrong_fields:
--> 159                 raise ValueError("dict contains fields not in fieldnames: "
    160                                  + ", ".join([repr(x) for x in wrong_fields]))
    161         return (rowdict.get(key, self.restval) for key in self.fieldnames)

*** NameError: name 'wrong_fieldnames' is not defined
['response', 'metrics']
['response', 'metrics']
<built-in method keys of dict object at 0x13006a540>
dict_keys(['id', 'query', 'persona', 'response', 'metrics'])
['response', 'metrics']
*** NameError: name 'quite' is not defined
